In [2]:
from tqdm import tqdm_notebook
from tqdm import tqdm

import json
import pandas as pd

from utils.openai_utils import LLMTripletExtractor
from utils.dynamic_index_utils import Aligner

import os
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

/home/jovyan/chepurova/Text2KG/venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open("musique_200_test.json", "r") as f:
    ds = json.load(f)

In [16]:
lines = []
with open('logs/logs_musique_50.jsonl', 'r') as f:
    for line in f:
        
        messages = json.loads(line)

        if messages['sample_id'] == '2hop__632870_22402':
            lines.append(messages)

In [17]:
lines

[{'sample_id': '2hop__632870_22402',
  'text_id': 0,
  'cost': 148.266275,
  '1-step triplets': [{'subject': 'Margate City Schools',
    'relation': 'instance of',
    'object': 'School district'},
   {'subject': 'Margate City Schools',
    'relation': 'educates',
    'object': 'Students in kindergarten through eighth grade'},
   {'subject': 'Margate City Schools',
    'relation': 'located in the administrative territorial entity',
    'object': 'Margate City'},
   {'subject': 'Margate City',
    'relation': 'located in the administrative territorial entity',
    'object': 'Atlantic County'},
   {'subject': 'Atlantic County',
    'relation': 'located in the administrative territorial entity',
    'object': 'New Jersey'},
   {'subject': 'New Jersey', 'relation': 'country', 'object': 'United States'},
   {'subject': 'Longport Borough',
    'relation': 'instance of',
    'object': 'Non-operating school district'},
   {'subject': 'Longport Borough',
    'relation': 'students attend',
    '

In [18]:
id2sample = {}
for elem in ds['data']:
    id2sample[elem['id']] = elem

In [19]:
id2sample['2hop__632870_22402']

{'id': '2hop__632870_22402',
 'paragraphs': [{'idx': 0,
   'title': 'Margate City Schools',
   'paragraph_text': "The Margate City Schools are a community public school district that serves students in kindergarten through eighth grade from Margate City, in Atlantic County, New Jersey, United States. Students from Longport Borough, a non-operating school district, jointly attend the district's schools.",
   'is_supporting': False},
  {'idx': 1,
   'title': 'Brigham Young University',
   'paragraph_text': 'The main campus in Provo, Utah, United States sits on approximately 560 acres (2.3 km2) nestled at the base of the Wasatch Mountains and includes 295 buildings. The buildings feature a wide variety of architectural styles, each building being built in the style of its time. The grass, trees, and flower beds on BYU\'s campus are impeccably maintained. Furthermore, views of the Wasatch Mountains, (including Mount Timpanogos) can be seen from the campus. BYU\'s Harold B. Lee Library (als